In [ ]:
# | default_exp create_vegetation_parameters

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [ ]:
# | export
import os
from pathlib import Path
from typing import Dict, List
from sureau_ecos_py.plant_utils import read_vegetation_file
from sureau_ecos_py.create_modeling_options import create_modeling_options
from sureau_ecos_py.create_stand_parameters import create_stand_parameters

In [ ]:
# | export

def create_vegetation_parameters(
    stand_parameters:Dict, # Dictionary created using the `create_stand_parameters` function
    file_path: Path = None,  # Path to a csv file containing lai_max, latitude and longitude values i.e path/to/parameter_values.csv
    list_of_parameters: List = None,  # A list containing the necessary input parameters instead of reading them in file. Will only be used if 'file_path' arguement is not provided
    modeling_options: Dict = None,  # Dictionary created using the `create_modeling_options` function
    sep: str = ";",  # CSV file separator can be ',' or ';'


) -> Dict:  # Dictionary containing parameters to run the model

    "Create a dictionary with the vegetation parameters to run \code{run.SurEauR}"

    # Assert parameters ---------------------------------------------------------

    # Make sure the file_path exist or is None
    assert file_path is None or os.path.exists(
        file_path
    ), f"Path: {file_path} not found, check spelling or set file_path = None"


    # Raise error if file_path and list_of_parameters both are missing
    if file_path is None and list_of_parameters is None:
        raise ValueError(
            "Both file_path and list_of_parameters are missing, provide one of them"
        )

    # Raise error if file_path and list_of_parameters both are provided
    if file_path is not None and list_of_parameters is not None:
        raise ValueError(
            "Both file_path and list_of_parameters are provided, only one of these two arguments should be used"
        )

    # Delete as soon the option is implemented
    if list_of_parameters is not None:
        raise ValueError(
            "Option for including list_of_parameters not implemented yet"
        )


    # Make sure that modeling_options is a dictionary
    assert isinstance(
        modeling_options, Dict
    ), f"modeling_options must be a dictionary not a {type(modeling_options)}"

    # Make sure that stand_parameters is a dictionary
    assert isinstance(
        stand_parameters, Dict
    ), f"stand_parameters must be a dictionary not a {type(stand_parameters)}"


    # Create vegetation_parameters from function inputs -------------------------
    vegetation_parameters = read_vegetation_file(file_path = file_path,
                                                 modeling_options = modeling_options,
                                                 sep = sep)

    # Compute stomatal response parameters for the sigmoid from P12_gs and P88_gs
    # provided in the file
    if modeling_options["stomatal_reg_formulation"] == "sigmoid":

        # Calculate p50
        if "p50_gs" not in vegetation_parameters:
            vegetation_parameters["p50_gs"] = (vegetation_parameters["p88_gs"] + vegetation_parameters["p12_gs"])/2
            vegetation_parameters["slope_gs"] = 100/(vegetation_parameters["p12_gs"] - vegetation_parameters["p88_gs"])


    # Get  maximum leaf area index of the stand (LAImax) from stand parameters
    #vegetation_parameters["lai_max"] = stand_parameters["lai_max"]




    #if modeling_options['']
    return vegetation_parameters

## Create vegetation parameters 

In [ ]:
modeling_options_dict_campbell = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="sigmoid",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="campbell",
)

In [ ]:
stand_params_from_csv = create_stand_parameters(
    file_path="./sample_data/stand_example.csv",
    lai_max=None,
    latitude=None,
    longitude=None,
    sep=";",
)
#stand_params_from_csv["lai_max"]

SchemaError: column 'plot_id' not in dataframe
   lai_max  latitude  longitude
0      4.5     48.73       6.23

In [ ]:
create_vegetation_parameters(
    file_path="./sample_data/vegetation_example_wide.csv",
    list_of_parameters = None,
    stand_parameters = stand_params_from_csv,
    modeling_options = modeling_options_dict_campbell,
    sep = ",",
)